In [8]:
import json
import numpy as np
import os
import pandas as pd
import urllib.request

# connect to poloniex's API
url = 'https://poloniex.com/public?command=returnChartData&currencyPair=USDT_BTC&start=1356998100&end=9999999999&period=300'

# parse json returned from the API to Pandas DF
openUrl = urllib.request.urlopen(url)
r = openUrl.read()
openUrl.close()
d = json.loads(r.decode())
df = pd.DataFrame(d)

original_columns=[u'close', u'date', u'high', u'low', u'open']
new_columns = ['Close','Timestamp','High','Low','Open']
df = df.loc[:,original_columns]
df.columns = new_columns
df.to_csv('./bitcoin2015to2017.csv',index=None)

In [10]:
import numpy as np
import pandas as pd
 
class PastSampler:
    '''
    Forms training samples for predicting future values from past value
    '''
     
    def __init__(self, N, K, sliding_window = True):
        '''
        Predict K future sample using N previous samples
        '''
        self.K = K
        self.N = N
        self.sliding_window = sliding_window
 
    def transform(self, A):
        M = self.N + self.K     #Number of samples per row (sample + target)
        #indexes
        if self.sliding_window:
            I = np.arange(M) + np.arange(A.shape[0] - M + 1).reshape(-1, 1)
        else:
            if A.shape[0]%M == 0:
                I = np.arange(M)+np.arange(0,A.shape[0],M).reshape(-1,1)
                
            else:
                I = np.arange(M)+np.arange(0,A.shape[0] -M,M).reshape(-1,1)
            
        B = A[I].reshape(-1, M * A.shape[1], A.shape[2])
        ci = self.N * A.shape[1]    #Number of features per sample
        return B[:, :ci], B[:, ci:] #Sample matrix, Target matrix

#data file path
dfp = 'bitcoin2015to2017.csv'

#Columns of price data to use
columns = ['Close']
df = pd.read_csv(dfp)
time_stamps = df['Timestamp']
df = df.loc[:,columns]
original_df = pd.read_csv(dfp).loc[:,columns]

In [19]:
file_name='bitcoin2015to2017_close.h5'

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
# normalization
for c in columns:
    df[c] = scaler.fit_transform(df[c].values.reshape(-1,1))

In [61]:
#Features are input sample dimensions(channels)
A = np.array(df)[:,None,:] #增加一个维度(x,y)=>(x,1,y)
original_A = np.array(original_df)[:,None,:]
time_stamps = np.array(time_stamps)[:,None,None]

In [63]:
#Make samples of temporal sequences of pricing data (channel)
NPS, NFS = 256, 16         #Number of past and future samples
ps = PastSampler(NPS, NFS, sliding_window=False)
B, Y = ps.transform(A)
input_times, output_times = ps.transform(time_stamps)
original_B, original_Y = ps.transform(original_A)

In [84]:
import h5py
with h5py.File(file_name, 'w') as f:
    f.create_dataset("inputs", data = B)
    f.create_dataset('outputs', data = Y)
    f.create_dataset("input_times", data = input_times)
    f.create_dataset('output_times', data = output_times)
    f.create_dataset("original_datas", data=np.array(original_df))
    f.create_dataset('original_inputs',data=original_B)
    f.create_dataset('original_outputs',data=original_Y)

/Users/yuchenshen/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [86]:
import pandas as pd
import numpy as numpy
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv1D, MaxPooling1D, LeakyReLU, PReLU
from keras.utils import np_utils
from keras.callbacks import CSVLogger, ModelCheckpoint
import h5py
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session


# Make the program use only one GPU
#os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
#os.environ['CUDA_VISIBLE_DEVICES'] = '1'
#os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))


with h5py.File(''.join(['bitcoin2015to2017_close.h5']), 'r') as hf:
    datas = hf['inputs'].value
    labels = hf['outputs'].value


output_file_name='bitcoin2015to2017_close_CNN_2_relu'

step_size = datas.shape[1]
batch_size= 8
nb_features = datas.shape[2]

epochs = 100

#split training validation
training_size = int(0.8* datas.shape[0])
training_datas = datas[:training_size,:]
training_labels = labels[:training_size,:]
validation_datas = datas[training_size:,:]
validation_labels = labels[training_size:,:]
#build model

# 2 layers
model = Sequential()


model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=20))
model.add(Dropout(0.5))
model.add(Conv1D( strides=4, filters=nb_features, kernel_size=16))

'''
# 3 Layers
model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=3, filters=8, kernel_size=8))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=8))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D( strides=2, filters=nb_features, kernel_size=8))
# 4 layers
model.add(Conv1D(activation='relu', input_shape=(step_size, nb_features), strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D(activation='relu', strides=2, filters=8, kernel_size=2))
#model.add(LeakyReLU())
model.add(Dropout(0.5))
model.add(Conv1D( strides=2, filters=nb_features, kernel_size=2))
'''

model.compile(loss='mse', optimizer='adam')
model.fit(training_datas, training_labels,verbose=1, batch_size=batch_size,validation_data=(validation_datas,validation_labels),
          epochs = epochs, 
          callbacks=[CSVLogger(output_file_name+'.csv', append=True),
                     ModelCheckpoint('./weights/'+output_file_name+'-{epoch:02d}-{val_loss:.5f}.hdf5',
                                     monitor='val_loss', verbose=1,mode='min')])


Train on 1147 samples, validate on 287 samples
Epoch 1/100
1147/1147 [==============================] - 1s 438us/step - loss: 0.0164 - val_loss: 0.0022

Epoch 00001: saving model to ./weights/bitcoin2015to2017_close_CNN_2_relu-01-0.00221.hdf5
Epoch 2/100
1147/1147 [==============================] - 0s 219us/step - loss: 0.0036 - val_loss: 0.0061

Epoch 00002: saving model to ./weights/bitcoin2015to2017_close_CNN_2_relu-02-0.00610.hdf5
Epoch 3/100
1147/1147 [==============================] - 0s 209us/step - loss: 0.0024 - val_loss: 0.0012

Epoch 00003: saving model to ./weights/bitcoin2015to2017_close_CNN_2_relu-03-0.00121.hdf5
Epoch 4/100
1147/1147 [==============================] - 0s 252us/step - loss: 0.0019 - val_loss: 8.6940e-04

Epoch 00004: saving model to ./weights/bitcoin2015to2017_close_CNN_2_relu-04-0.00087.hdf5
Epoch 5/100
1147/1147 [==============================] - 0s 215us/step - loss: 0.0015 - val_loss: 4.5111e-04

Epoch 00005: saving model to ./weights/bitcoin2015to201

1147/1147 [==============================] - 0s 272us/step - loss: 7.6331e-04 - val_loss: 3.5261e-04

Epoch 00042: saving model to ./weights/bitcoin2015to2017_close_CNN_2_relu-42-0.00035.hdf5
Epoch 43/100
1147/1147 [==============================] - 0s 242us/step - loss: 8.9781e-04 - val_loss: 6.0814e-04

Epoch 00043: saving model to ./weights/bitcoin2015to2017_close_CNN_2_relu-43-0.00061.hdf5
Epoch 44/100
1147/1147 [==============================] - 0s 207us/step - loss: 8.5802e-04 - val_loss: 2.1404e-04

Epoch 00044: saving model to ./weights/bitcoin2015to2017_close_CNN_2_relu-44-0.00021.hdf5
Epoch 45/100
1147/1147 [==============================] - 0s 239us/step - loss: 8.8149e-04 - val_loss: 0.0015

Epoch 00045: saving model to ./weights/bitcoin2015to2017_close_CNN_2_relu-45-0.00155.hdf5
Epoch 46/100
1147/1147 [==============================] - 0s 167us/step - loss: 9.7486e-04 - val_loss: 1.5238e-04

Epoch 00046: saving model to ./weights/bitcoin2015to2017_close_CNN_2_relu-46-0.000

1147/1147 [==============================] - 0s 221us/step - loss: 9.1275e-04 - val_loss: 2.0841e-04

Epoch 00082: saving model to ./weights/bitcoin2015to2017_close_CNN_2_relu-82-0.00021.hdf5
Epoch 83/100
1147/1147 [==============================] - 0s 221us/step - loss: 9.0395e-04 - val_loss: 4.0435e-04

Epoch 00083: saving model to ./weights/bitcoin2015to2017_close_CNN_2_relu-83-0.00040.hdf5
Epoch 84/100
1147/1147 [==============================] - 0s 219us/step - loss: 9.3097e-04 - val_loss: 1.3269e-04

Epoch 00084: saving model to ./weights/bitcoin2015to2017_close_CNN_2_relu-84-0.00013.hdf5
Epoch 85/100
1147/1147 [==============================] - 0s 213us/step - loss: 8.6639e-04 - val_loss: 2.4841e-04

Epoch 00085: saving model to ./weights/bitcoin2015to2017_close_CNN_2_relu-85-0.00025.hdf5
Epoch 86/100
1147/1147 [==============================] - 0s 221us/step - loss: 9.5344e-04 - val_loss: 3.3289e-04

Epoch 00086: saving model to ./weights/bitcoin2015to2017_close_CNN_2_relu-86-0

KeyboardInterrupt: 